In [1]:
from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError

from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf

from unstructured.staging.base import dict_to_elements

In [ ]:
client = UnstructuredClient(api_key_auth="us0KAUZs5xSGsGD64xkF1C7NOjigRk")

In [2]:
filename = "Additional_Data/SBC.pdf"
pdf_elements = partition_pdf(filename=filename, strategy="fast")

INFO: pikepdf C++ to Python logger bridge initialized


In [3]:
for element in pdf_elements[:10]:
    print(f"{element.category.upper()}: {element.text}")

HEADER: Summary of Benefits and Coverage: What this Plan Covers & What You Pay For Covered Services Insurance Company 1: Plan Option 1
HEADER: Coverage Period: 1/1/2017 – 12/31/2017 Coverage for: Family | Plan Type: PPO
NARRATIVETEXT: The Summary of Benefits and Coverage (SBC) document will help you choose a health plan. The SBC shows you how you and the plan would share the cost for covered health care services. NOTE: Information about the cost of this plan (called the premium) will be provided separately. This is only a summary. For more information about your coverage, or to get a copy of the complete terms of coverage, [insert contact information]. For general definitions of common terms, such as allowed amount, balance billing, coinsurance, copayment, deductible, provider, or other underlined terms see the Glossary. You can view the Glossary at www.[insert].com or call 1-800-[insert] to request a copy.
TITLE: Important Questions
NARRATIVETEXT: What is the overall deductible?
NARRA

In [4]:
with open(filename, "rb") as f:
    files=shared.Files(
        content=f.read(),
        file_name=filename,
    )

req = shared.PartitionParameters(
    files=files,
    strategy="hi_res",
    hi_res_model_name="yolox",
)

try:
    resp = s.general.partition(req)
    dld_elements = dict_to_elements(resp.elements)
except SDKError as e:
    print(e)

NameError: name 's' is not defined